# Tic Tac Toe Coin

To store state of tic tac toe game on Chia blockchain, we create an outer puzzle that wrap the inner tic tac toe puzzle. The coin puzzle outputs the conditions based on the output from the inner tic tac toe puzzle.

## Code
### [coin.clsp](./code/coin.clsp)
```lisp
;   MOD                         : the puzzle itself
;   PLAYER_ONE_INFO             : (PK, PUZZLE_HASH), public key and puzzle hash of the first player
;   PLAYER_TWO_INFO             : (PK, PUZZLE_HASH), public key and puzzle hash of the second player
;   CURRIED_TIC_TAC_TOE_PUZZLE  : tic tac toe puzzle with curried-in current board and next player
;   AMOUNT                      : coin amount (odd to be used with singleton top layer)
;   position                    : next play position as inner solution for curried tic tac toe puzzle

(mod (MOD PLAYER_ONE_INFO PLAYER_TWO_INFO CURRIED_TIC_TAC_TOE_PUZZLE amount position)
     ...
)     
```

## Generate PKs for player one and two
```sh
cdv inspect keys --random | grep public -i
cdv inspect keys --random | grep public -i
Public Key: ab7431b52af2ff59d4305893d705f11a07eb8ba979bbe8a01dc37e7616e9967bd9a787a45394983f2ef05c7b76cef2f0
Public Key: 9900d2dce0b44916b39715ba4d19f2d1c176c8d1c0001cec2f662e65d0fad984ef21969fe08264b7f42941836582dbdc
```


In [1]:
%%bash
chia version
cdv --version
python --version

1.5.0
cdv, version 1.0.8
Python 3.8.10


In [2]:
# chia libraries
from chia.types.blockchain_format.program import Program
from chia.wallet.puzzles import (p2_delegated_puzzle_or_hidden_puzzle)

# utils & tic tac toe helper code
import sys
sys.path.insert(0, "../../../shared")
from utils import (load_program, print_program, print_puzzle)

sys.path.insert(0, "./code")
import tic_tac_toe

# load puzzles
tic_tac_toe_puzzle = load_program("./code/tic-tac-toe.clsp", ["./code", "../../../shared"])
coin_puzzle = load_program("./code/coin.clsp", ["./code", "../../../shared"])

# prepare players info
player_one_pk = bytes.fromhex("ab7431b52af2ff59d4305893d705f11a07eb8ba979bbe8a01dc37e7616e9967bd9a787a45394983f2ef05c7b76cef2f0")
player_one_hash = p2_delegated_puzzle_or_hidden_puzzle.puzzle_for_pk(player_one_pk).get_tree_hash()
player_two_pk = bytes.fromhex("9900d2dce0b44916b39715ba4d19f2d1c176c8d1c0001cec2f662e65d0fad984ef21969fe08264b7f42941836582dbdc")
player_two_hash = p2_delegated_puzzle_or_hidden_puzzle.puzzle_for_pk(player_two_pk).get_tree_hash()
player_one_info = Program.to([player_one_pk, player_one_hash])
player_two_info = Program.to([player_two_pk, player_two_hash])

amount = 1023

In [3]:
def sim_play(board, player, position):

    # (mod (BOARD V pos)
    curried_tic_tac_toe_puzzle = tic_tac_toe_puzzle.curry(
            Program.to(board), 
            Program.to(player)
        ) 

    #(mod (MOD PLAYER_ONE_INFO PLAYER_TWO_INFO CURRIED_TIC_TAC_TOE_PUZZLE AMOUNT position)
    curried_coin_puzzle = coin_puzzle.curry(
        coin_puzzle,
        player_one_info,
        player_two_info,
        curried_tic_tac_toe_puzzle,
        amount)
    
    conditions = curried_coin_puzzle.run(Program.to([position]))
    coin_puzzle_hash = conditions.at("rfrf")
    print(coin_puzzle_hash)

    print()
    print_program(conditions)
    print()
    board_state, board = tic_tac_toe.play(curried_tic_tac_toe_puzzle, position)
    tic_tac_toe.print_board(board)
    print({
         0 : '',
        -1 : 'tie game',
        120: 'x wins',
        113: 'o wins'
    }[board_state])

    if board_state == 0:
        ## verify puzzle
        player = 'x' if player == 'o' else 'o'
        curried_tic_tac_toe_puzzle = tic_tac_toe_puzzle.curry(
                Program.to(board), 
                Program.to(player)
            )
        curried_coin_puzzle = coin_puzzle.curry(
            coin_puzzle,
            player_one_info,
            player_two_info,
            curried_tic_tac_toe_puzzle,
            amount)

        # new coin puzzle and curried_tic_tac_toe_coin_puzzle have to match
        assert curried_coin_puzzle.get_tree_hash() == coin_puzzle_hash
        return board, player
    return board, None

## Simulate Winning Board
1. Start with empty board.
2. `x` plays first.
3. `o` and `x` take turn until `x` wins.

In [4]:
board, player = sim_play([' '] * 9, 'x', 4)
board, player = sim_play(board, player, 0)
board, player = sim_play(board, player, 3)
board, player = sim_play(board, player, 1)
board, player = sim_play(board, player, 5)


a087c701c97a48f1d123eb6a90076abb25c868503960311c4c229314fbff40243d

((50 0xaba7ed288dd79189bec34698a3437fa7a45f801596d397a4f70081a0956dcdbe998388bfa758f4d49fa421ce3850a6d8 0x24644d734eb6f3bebcc3314494518cf5df9fb8e2614981b45bc753912c85ab19) (51 0x87c701c97a48f1d123eb6a90076abb25c868503960311c4c229314fbff40243d 1023))

   |   |   
---+---+---
   | x |   
---+---+---
   |   |   


a0d8167afd0187be8d88492d16824479198500c1274627ff8a8d180e70c02f857a

((50 0xaba7ed288dd79189bec34698a3437fa7a45f801596d397a4f70081a0956dcdbe998388bfa758f4d49fa421ce3850a6d8 0x0c822ff0ca88a532eff683ad7f46b486449a89b513f054c2d9835d22da45dad0) (51 0xd8167afd0187be8d88492d16824479198500c1274627ff8a8d180e70c02f857a 1023))

 o |   |   
---+---+---
   | x |   
---+---+---
   |   |   


a05957c200dccd6e079fafaec1429cc482cd119d810824d7b2901c81d833108076

((50 0xaba7ed288dd79189bec34698a3437fa7a45f801596d397a4f70081a0956dcdbe998388bfa758f4d49fa421ce3850a6d8 0x25a89c71e8510e60a26b484eac90002abd93de58d72cb85eafc706965ce0e0fa

## Simulate Tie Game
1. Start with empty board.
2. `x` plays first.
3. `o` and `x` take turn until the board is not playable.

In [5]:
board, player = sim_play([' '] * 9, 'x', 0)
board, player = sim_play(board, player, 1)
board, player = sim_play(board, player, 2)
board, player = sim_play(board, player, 4)
board, player = sim_play(board, player, 5)
board, player = sim_play(board, player, 6)
board, player = sim_play(board, player, 7)
board, player = sim_play(board, player, 8)
board, player = sim_play(board, player, 3)

a08acadc35b2d3fa063d1a8b153307cfe795393b9668ed5ea7d1c5d8c81674e99d

((50 0xaba7ed288dd79189bec34698a3437fa7a45f801596d397a4f70081a0956dcdbe998388bfa758f4d49fa421ce3850a6d8 0x78be85f051cd710988b61d7b9176dcbf7f3208c2a5c6a6ffa6688a6c631301f8) (51 0x8acadc35b2d3fa063d1a8b153307cfe795393b9668ed5ea7d1c5d8c81674e99d 1023))

 x |   |   
---+---+---
   |   |   
---+---+---
   |   |   


a0884e76517d3a9ccebbfd2ab563699582d5aeb0d77d4bb17e7c51c9ac9cae0494

((50 0xaba7ed288dd79189bec34698a3437fa7a45f801596d397a4f70081a0956dcdbe998388bfa758f4d49fa421ce3850a6d8 0xf0dbb49816f3792bb4952cbdfbf6502207d157d2b575b8b2b5e1de4e8057e56b) (51 0x884e76517d3a9ccebbfd2ab563699582d5aeb0d77d4bb17e7c51c9ac9cae0494 1023))

 x | o |   
---+---+---
   |   |   
---+---+---
   |   |   


a099ea176b80e76a0435a96354307d72f0196000ecf238998aacbfd9ce502ba216

((50 0xaba7ed288dd79189bec34698a3437fa7a45f801596d397a4f70081a0956dcdbe998388bfa758f4d49fa421ce3850a6d8 0x753165bbd1a199b62aa14a65ece45cb03588a1fffb0b4db3b77679fdf23d3f40

## Get values from curried puzzles

In [6]:
board = ['o', ' ', 'x', ' ', 'x', 'x', 'o', ' ', 'o']
board, player = sim_play(board, 'x', 1)

curried_tic_tac_toe_puzzle = tic_tac_toe_puzzle.curry(
    Program.to(board), 
    Program.to(player)
) 

curried_coin_puzzle = coin_puzzle.curry(
    coin_puzzle,
    player_one_info,
    player_two_info,
    curried_tic_tac_toe_puzzle)

curried_puzzle = tic_tac_toe.get_curried_puzzle_from_curried_coin_puzzle(curried_coin_puzzle)
print(tic_tac_toe.get_board_from_curried_puzzle(curried_puzzle))
print(tic_tac_toe.get_player_from_curried_puzzle(curried_puzzle))


a0a7815e8041f12488958b3337582bdbeaeb068548a042508459032fea0ecd3431

((50 0xaba7ed288dd79189bec34698a3437fa7a45f801596d397a4f70081a0956dcdbe998388bfa758f4d49fa421ce3850a6d8 0xf3198ddae9bc6bab45485796dac82816225f6d0cfb1ccbe350eb7ce0f3509efd) (51 0xa7815e8041f12488958b3337582bdbeaeb068548a042508459032fea0ecd3431 1023))

 o | x | x 
---+---+---
   | x | x 
---+---+---
 o |   | o 


[111, 120, 120, 32, 120, 120, 111, 32, 111]
o
